In [1]:
import hazm
import nltk
import pandas
import re 
import pickle
import arabic_reshaper

In [2]:
df = pandas.read_excel('final_books.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2824 entries, 0 to 2823
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     2824 non-null   object
 1   date      2824 non-null   object
 2   content   2441 non-null   object
 3   category  2824 non-null   object
 4   author    2824 non-null   object
 5   comments  2824 non-null   object
dtypes: object(6)
memory usage: 132.5+ KB


In [3]:
df.iloc[0]['content']

' تی دی جیکس یک کشیش، نویسنده، سخنران آمریکایی است. سمینار های تی دی جیکس اغلب انگیزشی و مذهبی است به همین دلیل او در ایران به سخنران انگیزشی سیاه پوست لقب گرفته است. در زندگینامه تی دی جیکس شاهد فعالیت\u200cهای مختلفی هستیم که هر کدام از آنها، خواهان تجربه و تلاش زیادی هستند. مواردی که اعلام شد تنها بخشی از فعالیت\u200cهای جیکس است. او هم اکنون کشیش خانه پاتر است و مخاطبان بسیاری دارد. موعظه\u200cهای\xa0 و سخنرانی\u200cها تی دی جیکس در کلیسا از شبکه\u200cهای تلویزیونی و رادیویی مختلف پخش می\u200cشوند. این کتاب گزیده ای از سخنرانی های آقای جیکز ، یکی از بهترین سخنران های انگیزشی در دنیا است. امید است با مطالعه دقیق و باور عمیق تغییری مثبت در زندگیتان ایجاد شود'

In [4]:
def remove_punctuations(text):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~،'''
    for char in text:
        if char in punctuations:
            text = text.replace(char, "")
    return text

In [5]:
import hazm

normalizer = hazm.Normalizer()
lemmatizer = hazm.Lemmatizer()
stop_words = hazm.stopwords_list()

tokens = []
lemmatized_tokens = []

for index, row in df.iterrows():
    content = row['content'] 
    norm_content = normalizer.normalize(str(content))
    norm_content_no_punctuations = remove_punctuations(norm_content)
    tokenized_sentences = hazm.sent_tokenize(norm_content_no_punctuations)
    content_tokenized_words = []
    for sentence in tokenized_sentences:
        try:
            tokenized_words = hazm.word_tokenize(sentence)
            filtered_words = [word for word in tokenized_words if word not in stop_words]
            content_tokenized_words.extend(filtered_words)
            for word in filtered_words:
                lemmatizedWord = lemmatizer.lemmatize(word)
                lemmatized_tokens.append((index, lemmatizedWord))
        except:
            print("error")  
    tokens.extend(content_tokenized_words)

In [6]:
def create_word_index(arr):
    word_index = {}
    for doc_id, word in arr:
        if word in word_index:
            word_index[word].append(doc_id)
        else:
            word_index[word] = [doc_id]
    return dict(sorted(word_index.items(), key=lambda x: len(x[1]), reverse=True))

In [7]:
word_index = create_word_index(lemmatized_tokens)

In [8]:
first_key, first_value = next(iter(word_index.items()))
print(first_key, first_value[0:100])

کتاب [0, 1, 2, 2, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 10, 11, 13, 13, 13, 13, 13, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 18, 18, 19, 21, 21, 21, 21, 22, 24, 25, 25, 26, 27, 27, 27, 29, 30, 30, 30, 34, 35, 37, 37, 37, 38, 39, 40, 41, 43, 43, 43, 44, 44, 45, 45, 45, 45, 45, 45, 46, 46, 46, 46, 47, 47, 47, 48, 48, 49, 50]


In [9]:
def sort_dict_values(d):
    sorted_dict = {}
    for k in d:
        sorted_dict[k] = sorted(set(d[k]), key=d[k].count, reverse=True)
    return sorted_dict

sorted_word_index = sort_dict_values(word_index)


In [10]:
sorted_word_index = sort_dict_values(word_index)
first_key, first_value = next(iter(sorted_word_index.items()))
print(first_key, first_value[0:20])

کتاب [502, 512, 428, 960, 1583, 103, 112, 188, 544, 8, 2213, 417, 988, 1116, 1170, 16, 460, 659, 1390, 1512]


In [11]:
words_count = []
for i in range(2824):
    words_count.append(len(str(df.iloc[i]['content']).split()))

In [12]:
tf = []
for i in range(2824):
    keyword = []
    for key in word_index:
        if words_count[i] != 0:
            keyword.append((key,word_index[key].count(i)/ words_count[i]))
        else:
            keyword.append((key,0))
    tf.append((i, keyword))

In [13]:
import math
idf = {}
for key in sorted_word_index:
    logarithm = math.log(2824/len(sorted_word_index[key]))
    idf[key] = logarithm

idf

{'کتاب': 0.6966945414987435,
 '–': 2.5568378687966318,
 '«': 1.5760086157849056,
 '»': 1.5863357299407552,
 'ایران': 1.7700423285073714,
 'زبان': 1.852339828567997,
 'سال': 1.5291773161008064,
 'کار': 1.7017426979493966,
 'زندگی': 1.7742090012022174,
 'تاریخ': 2.0542653867873617,
 '؛': 2.252177459810433,
 'استفاده': 1.8234167890987467,
 '؟': 2.3775650948520357,
 'شد#شو': 1.7953068301668533,
 'قرار': 1.7783931077247914,
 'داستان': 2.354922618102276,
 'کشور': 2.1169639810029257,
 'آموزش': 2.1933369597874997,
 'داشت#دار': 1.9619733179259422,
 'کرد#کن': 1.959457593328695,
 'انسان': 2.2158098156395583,
 'توانست#توان': 1.9080386786909949,
 'نام': 2.085123375147267,
 'صورت': 1.9494575099941116,
 'زمان': 2.0031102234864315,
 'عنوان': 1.977202038627767,
 'داد#ده': 2.0004889900065574,
 'دست': 2.0324065929748625,
 'علم': 2.3255087328959827,
 'مختلف': 2.125826668260771,
 'برنامه': 2.730973841004147,
 'مجموعه': 2.235482581238263,
 'انجام': 2.2829491184771866,
 'جهان': 2.2829491184771866,
 'نوشت#نوی

# correct

In [15]:
tfidf = []
# i => (dcoument id, [(word, tf), (word, tf), ...])
for i in tf:
    row = []
    # k => (word, tf_value)
    for k in i[1]:
        row.append({k[0]: k[1]*idf[k[0]]})
    tfidf.append((i[0], row))

In [16]:
tfidf[0]

(0,
 [{'کتاب': 0.005664183264217428},
  {'–': 0.0},
  {'«': 0.0},
  {'»': 0.0},
  {'ایران': 0.014390588036645297},
  {'زبان': 0.0},
  {'سال': 0.0},
  {'کار': 0.0},
  {'زندگی': 0.0},
  {'تاریخ': 0.0},
  {'؛': 0.0},
  {'استفاده': 0.0},
  {'؟': 0.0},
  {'شد#شو': 0.0},
  {'قرار': 0.0},
  {'داستان': 0.0},
  {'کشور': 0.0},
  {'آموزش': 0.0},
  {'داشت#دار': 0.0},
  {'کرد#کن': 0.0},
  {'انسان': 0.0},
  {'توانست#توان': 0.0},
  {'نام': 0.0},
  {'صورت': 0.0},
  {'زمان': 0.0},
  {'عنوان': 0.0},
  {'داد#ده': 0.0},
  {'دست': 0.0},
  {'علم': 0.0},
  {'مختلف': 0.03456628728879303},
  {'برنامه': 0.0},
  {'مجموعه': 0.0},
  {'انجام': 0.0},
  {'جهان': 0.0},
  {'نوشت#نویس': 0.0},
  {'nan': 0.0},
  {'توسط': 0.0},
  {'گرفت#گیر': 0.0},
  {'فرهنگ': 0.0},
  {'بود#باش': 0.0},
  {'روز': 0.0},
  {'فصل': 0.0},
  {'اطلاعات': 0.0},
  {'فارس': 0.0},
  {'نویسنده': 0.019423427129378824},
  {'مطالب': 0.0},
  {'دانست#دان': 0.0},
  {'بررسی': 0.0},
  {'اجتماع': 0.0},
  {'توجه': 0.0},
  {'اصل': 0.0},
  {'نمود#نما': 0.0},
  {'

# interface 

In [18]:
query = input("Enter your query: ")
query_words = query.split()

scores = []
#i => (document id, [(word, tfidf), (word, tfidf), ...])
for i in tfidf:
    score = 0
    for word in query_words:
        #i[0] => document id
        #j => (word, tfidf)
        print(i)
        for j in i[1]:
            if j[word] != None:
                score += i[1][word]
    scores.append((i[0], score))

scores.sort(key=lambda x: x[1], reverse=True)
scores

[{'کتاب': 0.005664183264217428}, {'–': 0.0}, {'«': 0.0}, {'»': 0.0}, {'ایران': 0.014390588036645297}, {'زبان': 0.0}, {'سال': 0.0}, {'کار': 0.0}, {'زندگی': 0.0}, {'تاریخ': 0.0}, {'؛': 0.0}, {'استفاده': 0.0}, {'؟': 0.0}, {'شد#شو': 0.0}, {'قرار': 0.0}, {'داستان': 0.0}, {'کشور': 0.0}, {'آموزش': 0.0}, {'داشت#دار': 0.0}, {'کرد#کن': 0.0}, {'انسان': 0.0}, {'توانست#توان': 0.0}, {'نام': 0.0}, {'صورت': 0.0}, {'زمان': 0.0}, {'عنوان': 0.0}, {'داد#ده': 0.0}, {'دست': 0.0}, {'علم': 0.0}, {'مختلف': 0.03456628728879303}, {'برنامه': 0.0}, {'مجموعه': 0.0}, {'انجام': 0.0}, {'جهان': 0.0}, {'نوشت#نویس': 0.0}, {'nan': 0.0}, {'توسط': 0.0}, {'گرفت#گیر': 0.0}, {'فرهنگ': 0.0}, {'بود#باش': 0.0}, {'روز': 0.0}, {'فصل': 0.0}, {'اطلاعات': 0.0}, {'فارس': 0.0}, {'نویسنده': 0.019423427129378824}, {'مطالب': 0.0}, {'دانست#دان': 0.0}, {'بررسی': 0.0}, {'اجتماع': 0.0}, {'توجه': 0.0}, {'اصل': 0.0}, {'نمود#نما': 0.0}, {'پرداخت#پرداز': 0.0}, {'مردم': 0.0}, {'ساده': 0.0}, {'دنیا': 0.020603768672830026}, {'آن': 0.0}, {'نوشته': 0.0

TypeError: list indices must be integers or slices, not str

In [ ]:
# # Save the word_dictionaries object to a file
# with open('index_dict.pkl', 'wb') as f:
#     pickle.dump(sorted_word_index, f)